In [1]:
import data
from data import DataLoader
import importlib
importlib.reload(data)
import torch
import torch.nn as nn
import torch.nn.functional as F

from gpt import GPTConfig

In [ ]:
data = DataLoader()
config = GPTConfig(vocab_size=50257, block_size=1024,
                   n_layer=12, n_head=12, n_embd=768)


In [18]:
print(data.inputs[0])
print(data.outputs[0])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13, 198, 198, 5962, 22307, 25, 198, 1639, 389, 477, 12939, 2138, 284, 4656, 621, 284, 1145, 680, 30, 198, 198, 3237, 25, 198, 4965, 5634, 13, 12939, 13, 198, 198, 5962, 22307, 25, 198, 5962, 11, 345, 760, 327, 1872, 385, 1526, 28599, 318, 4039, 4472, 284, 262, 661, 13, 198, 198, 3237, 25, 198, 1135, 760, 470, 11, 356, 760, 470, 13, 198, 198, 5962, 22307, 25, 198, 5756, 514, 1494, 683, 11, 290, 356, 1183, 423, 11676, 379, 674, 898, 2756, 13, 198, 3792, 470, 257, 15593, 30, 198, 198, 3237, 25, 198, 2949, 517, 3375, 319, 470, 26, 1309, 340, 307, 1760, 25, 1497, 11, 1497, 0, 198, 198, 12211, 22307, 25, 198, 3198, 1573, 11, 922, 4290, 13, 198, 198, 5962, 22307, 25, 198, 1135, 389, 17830, 3595, 4290, 11, 262, 1458, 1173, 1547, 922, 13, 198, 2061, 4934, 969, 5036, 896, 319, 561, 26958, 514, 25, 611, 484, 198, 19188, 7800, 514, 475, 262, 48713, 414, 11, 981, 340, 547, 198, 

In [74]:
one_tri = torch.tril(torch.ones(4,4))
print(one_tri)
m = torch.randint(0, 10, (4,4)).float()
print(m)

masked = m.masked_fill(one_tri==0, float('-inf'))
print(masked)

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[7., 4., 1., 7.],
        [5., 8., 4., 0.],
        [0., 6., 3., 1.],
        [8., 4., 4., 0.]])
tensor([[7., -inf, -inf, -inf],
        [5., 8., -inf, -inf],
        [0., 6., 3., -inf],
        [8., 4., 4., 0.]])


In [1]:
from qadata import *

In [2]:
qa_loader = QADataLoader(
    dataset_name="databricks/databricks-dolly-15k",
    batch_size=batch_size,
    block_size=block_size
)

# Test getting a batch (old style)
x, y = qa_loader.get_batch('train')
print(f"Batch shapes - Input: {x.shape}, Target: {y.shape}")
print(f"Vocabulary size: {vocab_size}")

# Decode and show a sample
sample_input = tokenizer.decode(x[0].tolist())
print(f"\nSample input (truncated):\n{sample_input[:200]}...")

# Modern training loop example
print("\n\nModern training loop example:")
train_loader, val_loader = qa_loader.get_loaders()
print(len(train_loader))
for i, (inputs, targets) in enumerate(train_loader):
    inputs = inputs.to(device)
    targets = targets.to(device)
    
    print(f"Batch {i}: Input shape {inputs.shape}, Target shape {targets.shape}")
    print(inputs[0], targets[0])
    
    if i >= 1:  # Just show first 3 batches
        break

Loading dataset: databricks/databricks-dolly-15k
Batch shapes - Input: torch.Size([32, 511]), Target: torch.Size([32, 511])
Vocabulary size: 32000

Sample input (truncated):
<s> ### USER: Classify each as either an amphibian, bird, or fish: salmon, trout, salamander, cardinal, owl, falcon, frog

### Answer: Salmon: Fish
Trout: Fish
Salamander: Amphibian
Cardinal: Bird
Owl...


Modern training loop example:
423
Batch 0: Input shape torch.Size([32, 511]), Target shape torch.Size([32, 511])
tensor([    1,   835,  3148,  1001, 29901,  3529,  2367,  3273, 15837,   310,
        15221, 29875, 18939, 29926,   350, 28690, 14113,  5073,  2729,   373,
         2400,    13,    13,  2277, 29937,   673, 29901, 15221, 29875, 18939,
        29926,   350, 28690, 14113,  6496,   297, 29871, 29896, 29929, 29906,
        29947,   297, 19628,  9188,  6474, 29889,   450, 15221, 29875, 18939,
        29926, 14113,  1196, 19623,   515, 15221, 11163, 29926,  6474, 26463,
         4726,   304,   278,  7392,   4